In [1]:
# import libraries
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline            

# 2 a)

### Function to create feature matrix and and label vectors
The function 'get_vectors' takes a filename as input and returns a matrix X containing feature vectors and vector y containing target labels.

In [2]:
def get_vectors(filename):
    try:
        f = open(filename, 'r')
    except OSError:
        print(f'{filename} could not be opened.\n')
        sys.exit()
        
    # initialize list to store feature and labels for training data
    features = []             
    labels = []
    
    with f:
        line = f.readline()
        while line != '':
            # strip newline and outer parenthesis
            line = line.strip('\n')
            line = line.strip('( )')
            
            # extrace label and append to labels list
            single_label = line.split('), ')[-1]
            labels.append(single_label)
            
            # extrace features and append to features list
            feat = line.split('), ')[0].split(', ')
            features.append(feat)
            
            # read next line
            line = f.readline()
        
        # create dataframe of features and append labels
        X = np.array(features, dtype = float, ndmin = 2)
        
        # convert labels list to array
        y = np.array(labels, dtype = str, ndmin = 2)
        
        return X, y.transpose()

### Distance Calculation

In [3]:
# calculates euclidean distance between training datapoints and test data point
def get_euclidean_distance(X_train, p):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    sum_of_squared_diff = np.zeros((n, 1), dtype = float)
    
    # use vectorization to get sum of squared difference
    for i in range(f_n):
        x_vector = X_train[:,i].reshape((n,1))
        sum_of_squared_diff = sum_of_squared_diff + (x_vector - p[i])**2
        
    # take sq root to get array of cartesianeuclidean distance
    euc_dist = np.sqrt(sum_of_squared_diff)
    
    return euc_dist

# calculates manhattan distance between training datapoints and test data point
def get_manhattan_distance(X_train, p):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    sum_of_abs_diff = np.zeros((n, 1), dtype = float)
    
    # use vectorization to get sum of squared difference
    for i in range(f_n):
        x_vector = X_train[:,i].reshape((n,1))
        sum_of_abs_diff = sum_of_abs_diff + abs(x_vector - p[i])
        
    # take sq root to get array of cartesianeuclidean distance
    man_dist = np.sqrt(sum_of_abs_diff)
    
    return man_dist



### KNN Implementation 

This function takes training data, prediction datapoint and number of neighbors as the argument and returns
a prediction class based on highest MAP value i.e. argmax_c P(class|<data>)

Key notes:
1. Cartesian/euclidean distance is used as a default distance measure.
2. The distance and labels of the first 'k' neighbors are printed.
3. MAP values of each class is calculated and stored in a dictionary with class labels as the key.
4. Tie breaker: If two or more classes have the same highest probability value, the first occuring class is returned.

In [4]:

def predict_class_with_knn(X_train, y_train, p, k, dist_type):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    if dist_type == 'euclidean':
        # get euclidean distance array
        dist_arr = get_euclidean_distance(X_train, p)
    elif dist_type == 'manhattan':
        # get euclidean distance array
        dist_arr = get_manhattan_distance(X_train, p)
    
    # concat with y_train labels and sort in ascending order of the distance
    dist_arr = np.concatenate((dist_arr, y_train), axis = 1)
    
    # sort wrt the euclidean distance(first col)
    dist_arr = dist_arr[dist_arr[:,0].argsort()]
    
    # save k nearest neighbors
    knn = dist_arr[0:k,:]
    
    # save labels of the k nearest neighbors as a list
    knn_labels = list(dist_arr[0:k,1]) 
    
    # calculate class probability
    class_probability = {}
    class_probability['Metal'] = knn_labels.count('Metal')/n
    class_probability['Ceramic'] = knn_labels.count('Ceramic')/n
    class_probability['Plastic'] = knn_labels.count('Plastic')/n
    
    max_probability = 0
    max_probability_class = max(class_probability, key = class_probability.get)
    
    for i in class_probability:
        
    # return the class with maximum probability and the probablity value
    return max(class_probability, key = class_probability.get), class_probability[max(class_probability, key = class_probability.get)]


### Provide training filename & prediction data point
The file must contain 1 datapoint per line in format (( height, diameter, weight, hue ), label ) which is similar to the format provided for the assignment

In [ ]:
# provide filename
filename = str(input('Enter file containing training data: '))

# provide prediction datapoint
h_p = float(input('Enter height: '))
d_p = float(input('Enter diameter: '))
w_p = float(input('Enter weight: '))
c_p = float(input('Enter hue/color: '))

# provide 'k' value
k = int(input('Enter value of k: '))

# save prediction datapont as a 1-D numpy array
p = np.array([h_p, d_p, w_p, c_p])

# get training data as vectors
X, y = get_vectors(filename)

# get prediction
preidicted_class, prob_value = predict_class_with_knn(X, y, p, k)
print(f"\nThe datapoint belongs to class '{preidicted_class}' with a probability of {prob_value:.4f}.")

## 2 c)

### Leave-one-out evaluation

This function takes the entire feature dataset(X) , correct data point labels(y), the number of neighbors to consider(k) and the type of distance to use for nearest neighbor calculation(dist_type) as arguments and returns the accuracy (total correct predictions/ total datapoints). 

The datapoint ot be left out and tested is chosen according to its index (i.e. item at index 0 is left out during the first iteration and item at index n-1 is left our during the last iteration).

In [5]:
def leave_one_out_validation(X, y, k, dist_type):
    
    # get number of training items and number of features
    n, f_n = X.shape
    
    # prediction labels generated by 'predict_class_with_knn' will be stored in this list
    predictions = []
    
    for i in range(n):
        X_train = np.delete(X, i, axis = 0)
        y_train = np.delete(y, i, axis = 0)
        X_test = X[i,:]
        pred, prob = predict_class_with_knn(X_train, y_train, X_test, k, dist_type)
        predictions.append(pred)
    
    # convert prediction list to numpy array
    predictions = np.array(predictions, dtype = str, ndmin = 2)
    predictions = predictions.reshape(predictions.shape[1], 1)
    
    # return accuracy
    return (np.sum(y == predictions))/n

### Enter filename for leave-one-out-evaluation

In [6]:
filename = str(input('Enter file for leave-one-out-evaluation data: '))

X, y = get_vectors(filename)

# initialize distionary to store accuracy values for different 'k' values
accuracy_euc = {}

Enter file for leave-one-out-evaluation data: Data\\2_c_d_e.txt


### K = 1, 3 and 5

In [9]:
# calculate accuracy for various values of 'k'
accuracy_euc[1] = leave_one_out_validation(X, y, 1, dist_type = 'euclidean')
accuracy_euc[3] = leave_one_out_validation(X, y, 3, dist_type = 'euclidean')
accuracy_euc[5] = leave_one_out_validation(X, y, 5, dist_type = 'euclidean')

In [10]:
accuracy_euc.items()

dict_items([(1, 0.425), (3, 0.38333333333333336), (5, 0.4083333333333333)])